In [1]:
import sys,os
import numpy as np, pandas as pd
import dill
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU as lrelu, PReLU
from keras.layers.core import Dropout
import tensorflow as tf

Using TensorFlow backend.


In [2]:
filename = 'mnist.pickle'
if os.path.exists(filename):
    with open(filename, 'rb') as f:
        mnist = dill.load(f)
else: 
    mnist = datasets.fetch_mldata('MNIST original', data_home='.')
    with open(filename, 'wb') as g:
        dill.dump(mnist, g)

In [3]:
n = len(mnist.data)
N = 10000 
indices = np.random.permutation(range(n))[:N]
X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

/Users/nagoyan/anaconda/envs/p3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
n_in = len(X[0])
n_hidden=200
n_out = len(Y[0])
print(n_in, n_hidden, n_out)

784 200 10


### TensorFlow

In [5]:
x= tf.placeholder(tf.float32, shape=[None, n_in])
t=tf.placeholder(tf.float32, shape=[None, n_out])
keep_prob = tf.placeholder(tf.float32) # probability for keeping


In [6]:
#--> input and hidden layers

W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0)+b0)
h0_drop = tf.nn.dropout(h0, keep_prob)

#--> hidden leyers

W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0_drop, W1)+b1)
h1_drop = tf.nn.dropout(h1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
h2 = tf.nn.relu(tf.matmul(h1_drop, W2)+b2)
h2_drop = tf.nn.dropout(h2, keep_prob)

#--> hidden and output layers

W3 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h2_drop, W3)+b3)

cross_entropy = - tf.reduce_sum(t * tf.log(y) + ( 1 - t ) * tf.log(1-y))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
epochs = 100
batch_size = 100
n_batches = N // batch_size

print(n_in, n_hidden, n_out)
print(epochs, batch_size, n_batches)

784 200 10
100 100 100


In [8]:

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={x: X_[start:end], t:Y_[start:end], keep_prob:0.5})
        
accuracy_rate = accuracy.eval(session=sess, feed_dict={x: X_test, t:Y_test, keep_prob:1.0})

print('accuracy : ', accuracy_rate)

accuracy :  0.9


In [9]:
print('accuracy : ', accuracy_rate)

accuracy :  0.9


### Keras

In [10]:
funcs = ['tanh', 'relu', 'softmax', 'sigmoid']
activation_func='relu'
alpha = 0.01 # parameter for LeakyReLU

model = Sequential()

model.add(Dense(n_hidden, input_dim=n_in))
#model.add(Activation(PReLU()))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
#model.add(Activation(PReLU()))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
#model.add(Activation(PReLU()))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_out))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])

In [11]:
epochs = 100
batch_size = 100
n_batches = N // batch_size

print(n_in, n_hidden, n_out)
print(epochs, batch_size, n_batches)

784 200 10
100 100 100


In [12]:

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)


Epoch 1/100
8000/8000 [==============================] - 2s 216us/step - loss: 2.5869 - acc: 0.1636

In [13]:
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

2000/2000 [==============================] - 0s 135us/step
[0.36021743318438532, 0.90349999999999997]
